In [1]:
import os
import sys
import zipfile 
import pandas as pd
from upload import retrieve_file
import warnings
from stim_adj import stim_adj
import autograd.numpy as np 
from autograd import grad
from scipy import optimize


In [2]:
class ParameterRecovery:
    def __init__(self, known_params, unknown_params, dt = 0.0125, neuron_type= 'L5PC', num_ap = 1, V_data = None, I_data = None, t_data = None, method = 'CG', bounds = {}): 
        '''
        For known parameters of the Hodgkin Huxley set the upper and lower bounds equal to that float
        Objective: 
        
        Args: 
            known_params (dict): known parameters and their values
            unknown_parameters (dict): unknown parameters and their intial guesses -- if different than standard
            neuron_types (str): 
            num_ap (int):
            V_data ():
            I_data ():
            t_data ():
            bounds ():
            method (str): with bo
        
        Returns: 
            '''
        self.neuron_type = neuron_type
        self.num_ap = num_ap
        self.known_params = known_params
        self.unknown_params = unknown_params
        self.params = {**self.known_params, **self.unknown_params}
        self.HH_var = ['g_Na', 'g_K', 'g_L', 'E_Na', 'E_K', 'E_L', 'C_m', 'm', 'h', 'n']
        self.param_list = list(self.params.keys())
        self.missing = list(set(self.HH_var) - set(self.param_list))
        self.unknown_keys = list(self.unknown_params.keys()) + self.missing
        
        self.dt = dt
    
        #default parameter setting HH model parameters
        
        self.g_Na = 120.0
        self.g_K = 36.0
        self.g_L = 0.3
        self.E_Na = 50.0
        self.E_K = -85.0
        self.E_L = -90
        self.C_m = 1.0
        self.m = 0.05
        self.n = 0.6
        self.h =  0.32
        self.standard_params = {'g_Na' :  self.g_Na , 'g_K' : self.g_K, 'g_L': self.g_L, 'E_Na': self.E_Na, 'E_K': self.E_K, 'E_L': self.E_L, 'C_m': self.C_m, 'm': self.m, 'h': self.h, 'n': self.n }
        #manual input variables 
        self.V_data = V_data
        self.I_data = I_data 
        self.t_data = t_data 
        
        #creating instance to load specific neuron data
        self.inst_file = retrieve_file(self.neuron_type, self.num_ap, self.V_data, self.I_data, self.t_data)
        
        #initialize variables dependent on ground truth data (from upload.py)
        instance = self.inst_file
        initialized = instance.load()
    
            
        #if manual entry is not choosen, upload from initialized 
        if self.V_data is None:
            self.V_data = initialized[0]
        else: 
            self.V_data = V_data
        if 'stim' not in self.unknown_params: 
            if self.I_data is None:
                self.I_data = initialized[1]
        if self.t_data is None: 
            self.t_data = initialized[2]
                   
        self.V0 = self.V_data[0]
        self.data_steps = self.t_data[1] - self.t_data[0]

        #optimizaton parameters
        #check that if bounds are given the optimal method is given 
        
        self.bounds = bounds
        if 'stim' in self.unknown_params and self.bounds is None: 
                warnings.warn("No bounds given, recommended methods to be CG or BFGS") #also want that no known values given 
                

    def assign_parameters(self):
        if 'stim' in self.unknown_params: 
            warnings.warn('If input stimuli is unknown, can only invert for features of the stimuli, all other parameters will be assumed to be known and set to default')
        for key, value in self.known_params.items():
            globals()[key] = value
        for key, value in self.unknown_params.items():
            globals()[key] = value
        missing_values = list(set(self.HH_var) - set(self.param_list))
        str_missing = ' '.join(missing_values)
        warnings.warn(f'The following variables were not defined or given an initial guess  {str_missing}. We will assume default values for an initial guess and that these values are unknown')
        
        #update parameters based on given known values
        
        return self.param_list
    
    def define_bounds(self):
        '''
        defines the bounds for optimiatoin setting upper and lower limits equal to known values
        '''
        if 'stim' in self.unknown_params: 
            optim_bounds = self.bounds
        else:
            optim_bounds = []
            for item in self.HH_var:
                if item in self.bounds: 
                    item_bound = self.bounds[item]
                elif item in self.known_params:
                    item_bound = (self.known_params[item], self.known_params[item])
                else: 
                    item_bound = (-float('inf'), float('inf'))
                optim_bounds.append(item_bound)
        return optim_bounds
                    
                    
    def def_guess_params(self):
        stim_guess = []
        HH_guess = []
        if 'stim' in self.unknown_params: 
            stim_guess.append(self.unknown_params['stim'])
            for item in self.HH_var:
                if item in self.known_params: 
                    HH_guess.append(self.known_params[item])
                else: 
                    HH_guess.append(self.standard_params[item])
        else: 
            for item in self.HH_var:
                if item in self.unknown_params: 
                    stim_guess.append(self.unknown_params[item])
                elif item in self.known_params: 
                    stim_guess.append(self.known_params[item])
                else: 
                    stim_guess.append(self.standard_params[item])
        return stim_guess, HH_guess
                         
    def adj_impulse(self): 
        params = self.def_guess_params()
        stim_params = params[0]
        HH_params = params[1]
        print(stim_params)
        print(stim_params[0])
        impulse_inst= stim_adj(self.V0, self.V_data, self.t_data, self.data_steps, self.dt, HH_params,  stim_params[0][0], stim_params[0][1], stim_params[0][2])
        return impulse_inst
    
    def optimizer_imp(self):
        instance = adj_impulse()
        x = instance.__cost(I_params)
        optim = optimize.minimize(instance.__cost, GUESS, args = (m, n, h), jac = grad_AD, method = self.method)
   # def optimizer(self):
       # if 'stim' in 
       # optimize.minimize(fm_cost, initial_guess, args = (self.I_data, self.V0, self.dt, self.t, datasteps, self.V_data))
        #create instance
        #call function?? 
        
    
    #def _minimize(self):
    
    def graph(self):
        plt.figure()
        plt.plot(self.t, self.V_final, ls = 'dotted', label = 'optimized spike train')
        plot.plot(self.t, self.V_data, label = 'target spike train')
        plt.xlabel('Time')
        plt.ylabel('Voltage')
        plt.legend()
        

In [3]:
y = [(1, 2, 3)]
print(y[0][0])

1


In [4]:
test = ParameterRecovery({'Na': 120, 'g_Na' : 5}, {'K': 5, 'stim': (5.0, 2.0, 152.525)}, bounds = {'E_L': (0, 10)})
test.assign_parameters()
test.unknown_keys
x = test.define_bounds()
y = test.def_guess_params()
print(y)
test.adj_impulse()
print(x)
type(test)






Default choosen of 1 Action Potential
([(5.0, 2.0, 152.525)], [5, 36.0, 0.3, 50.0, -85.0, -90, 1.0, 0.05, 0.32, 0.6])
[(5.0, 2.0, 152.525)]
(5.0, 2.0, 152.525)
{'E_L': (0, 10)}


/var/folders/y5/qy3mtncn089941sxdcnsvzn80000gn/T/ipykernel_25826/3496627311.py:82: UserWarning: If input stimuli is unknown, can only invert for features of the stimuli, all other parameters will be assumed to be known and set to default
  warnings.warn('If input stimuli is unknown, can only invert for features of the stimuli, all other parameters will be assumed to be known and set to default')
/var/folders/y5/qy3mtncn089941sxdcnsvzn80000gn/T/ipykernel_25826/3496627311.py:89: UserWarning: The following variables were not defined or given an initial guess  n g_L g_K E_K E_L E_Na h m C_m. We will assume default values for an initial guess and that these values are unknown
  warnings.warn(f'The following variables were not defined or given an initial guess  {str_missing}. We will assume default values for an initial guess and that these values are unknown')


__main__.ParameterRecovery

In [5]:
test.inst_file
test.inst_file.load()
x = test.inst_file.load()

print(x[0])

Default choosen of 1 Action Potential
Default choosen of 1 Action Potential
[-80.       -80.002106 -80.003914 ... -78.53216  -78.53172  -78.53129 ]
